In [76]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [77]:
class SM:
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s
        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

class Accumulator(SM):
    #startState = 0
    
    def __init__(self, initialValue):
        self.startState = initialValue

    def getNextValues(self, state, inp):
        return state + inp, state + inp

In [78]:
a = Accumulator()
a.start()

TypeError: __init__() missing 1 required positional argument: 'initialValue'

In [79]:
a = Accumulator(2)
a.start()

In [80]:
a.step(3)

5

In [81]:
a = Accumulator(5)
a.transduce([5,3])

[10, 13]

In [77]:
class SM:
    
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s
        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

class Accumulator(SM):
    #startState = 0
    
    def __init__(self, initialValue):
        self.startState = initialValue

    def getNextValues(self, state, inp):
        return state + inp, state + inp

In [95]:
class SM:
    
    initialized = False
    errorStatus = 0
    
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s
        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

In [110]:
class SpectralCalibrationMachine(SM):
    
    startState = 'waiting'
    
    def __init__(self):
        print('Launch GUI')
        print('Start by Initialization')
        self.initialize()
        
    def initialize(self):
        print('Run Initialization Procedure')
        self.intialized = True
            
    def generateOutput(self,state):
        if state == 'initializing':
            return 'initialize'
        elif state == 'collecting':
            return 'collect'
        elif state == 'compressing':
            return 'compress'
        elif state == 'storing':
            return 'write to disk'
        else:
            return 'wait' 
        
    # Inputs to consider: initStatus, errorStatus, rawDataCompressReady, rawDataCompressed, dataWriteReady, dataWritten
    def getNextValues(self, state, inp):
        #(initStatus, errorStatus, rawDataCompressReady, rawDataCompressed, dataWriteReady, dataWritten) = inp
        #pdb.set_trace()
        if self.errorStatus == True:
            nextState = 'error' 
        elif state == 'waiting' and inp == 'initialize':
            nextState = 'initializing'
        elif state == 'waiting' and self.init_status == True and inp == 'collect':
            nextState = 'collecting'
        elif state == 'waiting' and self.data_in_queue == True and inp == 'compress':
            nextState = 'compressing'
        elif state == 'waiting' and self.data_ready_to_store == True and inp == 'write_to_disk':
            nextState = 'storing'
        else:
            nextState = 'error'
            
        return (nextState, self.generateOutput(nextState))

In [111]:
scm = SpectralCalibrationMachine()

Launch GUI
Start by Initialization
Run Initialization Procedure


In [101]:
scm.start()

In [102]:
inp = (False, False, False, False, False, False)
scm.transduce([inp])

['wait']

In [109]:
scm.state

AttributeError: 'SpectralCalibrationMachine' object has no attribute 'state'

## Nomenclature
- Complete data collection procedure, from init to writing is referred to as a "run".   
- Each step in a run is an "event".  
- Where a run is in the sequence of events is it's "state"
- To move between states requires "triggers"

## States are
* waiting: in between events; triggered by conclusion of steps.  Options while waiting include
    * set parameters (wavelength/steps, integration time, devices to include in the initialization, compression?, write results?, display in real time?)
    * initialize
    * run all
* initializing (complete and between runs): triggered by initialize command (at any time, but warning if initialize already happened) will initialize all devices with checked boxes in GUI. Returns dictionary of device metadata
* collecting: triggered by collection command (at end sets self.data_in_queue = True)
* compressing: triggered by compression commmand (at end sets self.data_ready_to_store = True) 
* writing: triggered by writing command
* fubar: triggered by an error in any of the steps
* interupt: is this a state or a command?  

In [28]:
import pdb
import numpy as np
class SM:
    
    startState = 'waiting'
    init_status = False
    data_in_queue = False
    data_ready_to_store = False
    
    errorStatus = 0
    
    def start(self):
        self.state = self.startState

    def step(self,inp):
        (s,o) = self.getNextValues(self.state,inp)
        self.state = s

        return o

    def transduce(self, inputs):
        self.start()
        return [self.step(inp) for inp in inputs]

In [51]:
class SpectralCalibrationMachine(SM):
    
    def __init__(self):
        print('Start by Initializing')
               
        self.errorStatus = self.initialize()
        #pdb.set_trace()

    def initialize(self):
        print('Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict')
        
        self.start()
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.init_status = True
        return 'waiting', 'Initialized without errors' #errors 
    
    def collect_data(self):
        print('Collecting Data')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_in_queue = True
        return 'waiting', 'Data Collected' #errors 
    
    def compress_data(self):
        print('Compressing Data')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_ready_to_store = True
        return 'waiting', 'Data Compressed' #errors 
    
    def write_data_to_disk(self):
        print('Writing Data to Disk')
        errors = 0
        if np.sum(errors > 0):
            return 
        else:
            self.data_ready_to_store = True
        return 'waiting', 'Data Written to Disk' #errors 
    
    def generateOutput(self,state):
        if state == 'initializing':
            return self.initialize()
        elif state == 'collecting':
            return self.collect_data()
        elif state == 'compressing':
            return self.compress_data()
        elif state == 'storing':
            return self.write_data_to_disk()
        else:
            return 'wait'
 
    '''def getNextValues(self, state, inp):
        if self.errorStatus == True:
            nextState = 'error'      
        elif state == 'waiting' and inp == 'initialize':
            nextState = 'initializing'
        elif state == 'waiting' and self.init_status == True and inp == 'collect':
            nextState = 'collecting'
        elif state == 'waiting' and self.data_in_queue == True and inp == 'compress':
            nextState = 'compressing'
        elif state == 'waiting' and self.data_ready_to_store == True and inp == 'write_to_disk':
            nextState = 'storing'
        else:
            nextState = 'error'
    '''
    
    def getNextValues(self, state, dict_in):
        if self.errorStatus == True:
            nextState = 'error'      
        elif state == 'waiting' and 'initialize' in dict_in:
            nextState = 'initializing'
        elif state == 'waiting' and self.init_status == True and 'collect' in dict_in:
            nextState = 'collecting'
        elif state == 'waiting' and self.data_in_queue == True and 'compress' in dict_in:
            nextState = 'compressing'
        elif state == 'waiting' and self.data_ready_to_store == True and 'write_to_disk' in dict_in:
            nextState = 'storing'
        else:
            nextState = 'error'
        return (self.generateOutput(nextState))

In [52]:
scm2 = SpectralCalibrationMachine()

Start by Initializing
Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict


In [56]:
scm2.transduce([{'initialize': [0,2,4]},{'collect': [0,2,4]},{'compress': [0,2,4]},{'write_to_disk': [0,2,4]}])

Initialize each device, one at a time.  Each Returns Flag and Metadata, stored in a Dict
Compressing Data
Writing Data to Disk


['Initialized without errors',
 'Data Collected',
 'Data Compressed',
 'Data Written to Disk']

In [ ]:
scm2.transduce(['initialize','collect','compress','write_to_disk'])